In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

In [ ]:
from pathlib import Path
import sys
sys.path.insert(0, "..")

import cv2
import dlib
import numpy as np
from tqdm import tqdm

from data import plot, Process300VW
from demo import extract
from utils import constants, data_utils, personal_constants

In [ ]:
input_path = Path('/home/klaus/dev/5_master/5204PRAI6Y-project-ai/data/local_data/stijn')
temp_path = input_path / 'temp'
output_path = input_path / f'processed_dim{constants.DATASET_300VW_IMSIZE}'
FPS = 30

In [ ]:
if not temp_path.exists():
    temp_path.mkdir(parents=True, exist_ok=True)
    video_path = input_path / 'video.webm'
    video = cv2.VideoCapture(str(video_path))
    counter = 1
    success, image = video.read()
    while success == 1:
        frame_output_path = temp_path / f'{counter:06d}.jpg'
        cv2.imwrite(str(frame_output_path), image, [int(cv2.IMWRITE_JPEG_QUALITY), constants.DATASET_300VW_IMAGE_QUALITY])
        counter += 1
        success, image = video.read()

In [ ]:
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('./local_data/shape_predictor_68_face_landmarks.dat')

In [ ]:
n_frames = len(list(temp_path.glob('*.jpg')))
print(n_frames)

In [ ]:
frame_output_dir = output_path / 'images'
frame_output_dir.mkdir(parents=True, exist_ok=True)
landmarks = np.empty(
    (n_frames, constants.DATASET_300VW_N_LANDMARKS, 2)
)

counter = 1
for image_input_path in tqdm(sorted(list(temp_path.glob('*.jpg')))):
    image = cv2.imread(str(image_input_path))
    bounding_boxes = detector(image, 1)
    dlib_landmarks = [
        predictor(image, rectangle).parts() for rectangle in bounding_boxes
    ]
    if len(dlib_landmarks) != 1 or len(dlib_landmarks[0]) != constants.DATASET_300VW_N_LANDMARKS:
        continue
    image_landmarks = np.asarray([(lm.x, lm.y) for lm in dlib_landmarks[0]], dtype=float)
    
    image_box = data_utils.landmarks_to_box(image_landmarks, image.shape)
    extracted_image = data_utils.extract(image, image_box)
    extracted_landmarks = data_utils.offset_landmarks(image_landmarks, image_box)

    output = Process300VW._rescale_image(extracted_image)
    output_landmarks = data_utils.rescale_landmarks(
        extracted_landmarks, extracted_image.shape, constants.DATASET_300VW_IMSIZE
    )
    
    if counter == 1:
        plot(image, image_landmarks, image_box)
        plot(output, output_landmarks)
    
    frame_output_path = frame_output_dir / f'{counter:06d}.jpg'
    cv2.imwrite(str(frame_output_path), output, [int(cv2.IMWRITE_JPEG_QUALITY), constants.DATASET_300VW_IMAGE_QUALITY])
    counter += 1
    
    landmarks[counter-1] = output_landmarks
    
landmarks = landmarks[:counter]
np.save(str(output_path / 'annotations.npy'), landmarks)